In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import os
from sklearn.metrics import log_loss

In [7]:
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1
data=pd.concat([train,test],ignore_index=True)

continuous=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
target=["target"]
id=["uid"]
train.columns
categorical=['workclass','marital-status', 'occupation', 'relationship', 'race','sex', 'native-country','education']
dummies=pd.get_dummies(data[categorical])
dcols=dummies.columns
print(data.shape)
data=data.merge(dummies,how='left',left_index=True,right_index=True)
print(data.shape)
train=data[data["target"]!=-1]
test=data[data["target"]==-1]
len(train),len(test)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
X=train.drop(target+id+categorical,axis=1).values
y=train.target.values

scorer=make_scorer(log_loss, greater_is_better=False)

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=5,n_jobs=-1 ,scoring=scorer)
grid_search_cv.fit(X, y)


(48842, 16)
(48842, 118)
Fitting 5 folds for each of 294 candidates, totalling 1470 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 1470 out of 1470 | elapsed:   32.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
  

In [9]:
def evaluate_mdl(X,y):
    
    scorer=make_scorer(log_loss, greater_is_better=False)

    params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
    grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3,n_jobs=-1 ,scoring=scorer)
    grid_search_cv.fit(X, y)
    NFOLDS=5
    skf = StratifiedKFold(n_splits=NFOLDS)
    
    preds=np.zeros((X_test.shape[0],))
    losses=[]
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        # print(X_train.shape,X_val.shape)
        clf=grid_search_cv.best_estimator_
            
        clf.fit(X_train,y_train)
        pval=clf.predict_proba(X_val)[:,1]
        loss=log_loss(y_val,pval)
        losses.append(loss)
    print(np.mean(losses))
    return np.mean(losses)
evaluate_mdl(X,y)

Fitting 3 folds for each of 294 candidates, totalling 882 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   15.3s finished
0.32860536226942266


0.32860536226942266

In [13]:
continuous=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
target=["target"]
id=["uid"]
categorical=['workclass','marital-status', 'occupation', 'relationship', 'race','sex', 'native-country','education']
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1
data=pd.concat([train,test],ignore_index=True)
dummies=pd.get_dummies(data[categorical])
dcols=dummies.columns
print(data.shape)
data=data.merge(dummies,how='left',left_index=True,right_index=True)
for col in categorical:
    enc=LabelEncoder()
    data[col]=enc.fit_transform(data[col].values.reshape(-1,1)).flatten()
train=data[data["target"]!=-1]
test=data[data["target"]==-1]
X=train.drop(target+id,axis=1).values
y=train.target.values
"both dummy and label"
evaluate_mdl(X,y)

(48842, 16)
Fitting 3 folds for each of 294 candidates, totalling 882 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 859 out of 882 | elapsed:   17.0s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   17.3s finished
0.3156478721463144


0.3156478721463144

In [81]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,QuantileTransformer,PowerTransformer,Normalizer,MaxAbsScaler,KBinsDiscretizer,RobustScaler
continuous=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
target=["target"]
id=["uid"]
categorical=['workclass','marital-status', 'occupation', 'relationship', 'race','sex', 'native-country','education']
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1
data=pd.concat([train,test],ignore_index=True)
cnts=data[categorical]["native-country"].value_counts()
smallcnts=list(cnts[cnts<100].index)
data.loc[data["native-country"].isin(smallcnts),"native-country"]="others"
dummies=pd.get_dummies(data[categorical])
dcols=dummies.columns
print(data.shape)
data=data.merge(dummies,how='left',left_index=True,right_index=True)
for col in categorical:
    enc=LabelEncoder()
    data[col]=enc.fit_transform(data[col].values.reshape(-1,1)).flatten()

data["net-cap"]=data["capital-gain"]-data["capital-loss"]

# for col in continuous:
#     sc= StandardScaler()
#     data[col]=sc.fit_transform(data[col].values.reshape(-1,1)).flatten()
train=data[data["target"]!=-1]
test=data[data["target"]==-1]
X=train.drop(target+id,axis=1).values
y=train.target.values
"both dummy and label + scaling"
evaluate_mdl(X,y)

(48842, 16)
Fitting 3 folds for each of 294 candidates, totalling 882 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 859 out of 882 | elapsed:   13.9s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   14.3s finished
0.31557310671878147


0.31557310671878147

In [61]:
for c in categorical:
    print(c,data[c].nunique())
    

workclass 9
marital-status 7
occupation 15
relationship 6
race 5
sex 2
native-country 17
education 16


In [90]:
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1
data=pd.concat([train,test],ignore_index=True)
data["marital-status"].value_counts()
data["native-country"].value_counts().index

Index([' United-States', ' Mexico', ' ?', ' Philippines', ' Germany',
       ' Puerto-Rico', ' Canada', ' El-Salvador', ' India', ' Cuba',
       ' England', ' China', ' South', ' Jamaica', ' Italy',
       ' Dominican-Republic', ' Japan', ' Guatemala', ' Poland', ' Vietnam',
       ' Columbia', ' Haiti', ' Portugal', ' Taiwan', ' Iran', ' Greece',
       ' Nicaragua', ' Peru', ' Ecuador', ' France', ' Ireland', ' Thailand',
       ' Hong', ' Cambodia', ' Trinadad&Tobago', ' Yugoslavia',
       ' Outlying-US(Guam-USVI-etc)', ' Laos', ' Scotland', ' Honduras',
       ' Hungary', ' Holand-Netherlands'],
      dtype='object')

In [226]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,QuantileTransformer,PowerTransformer,Normalizer,MaxAbsScaler,KBinsDiscretizer,RobustScaler
continuous=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
target=["target"]
id=["uid"]
categorical=['workclass','marital-status', 'occupation', 'relationship', 'race','sex', 'native-country','education']
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1
data=pd.concat([train,test],ignore_index=True)
cnts=data[categorical]["native-country"].value_counts()
smallcnts=list(cnts[cnts<100].index)
data.loc[data["native-country"].isin(smallcnts),"native-country"]="others"
dummies=pd.get_dummies(data[categorical])
dcols=dummies.columns
print(data.shape)
data=data.merge(dummies,how='left',left_index=True,right_index=True)
for col in categorical:
    enc=LabelEncoder()
    data[col]=enc.fit_transform(data[col].values.reshape(-1,1)).flatten()


data["net-cap"]=data["capital-gain"]-data["capital-loss"]
# for col in continuous:
#     data[col]=(data[col].values/100).round()

# for col in continuous:
#     sc= StandardScaler()
#     data[col]=sc.fit_transform(data[col].values.reshape(-1,1)).flatten()
train=data[data["target"]!=-1]
test=data[data["target"]==-1]
X=train.drop(target,axis=1).values
y=train.target.values
"both dummy and label + scaling"
evaluate_mdl(X,y)

(48842, 16)
Fitting 3 folds for each of 294 candidates, totalling 882 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 859 out of 882 | elapsed:   15.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   15.5s finished
0.3173811870523929


0.3173811870523929

In [193]:

scorer=make_scorer(log_loss, greater_is_better=False)

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4],\
'min_samples_leaf':[1, 5, 10, 20, 50, 100]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3,n_jobs=-1 ,scoring=scorer)
grid_search_cv.fit(X, y)
grid_search_cv.best_estimator_

Fitting 3 folds for each of 1764 candidates, totalling 5292 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 2428 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 3528 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done 4828 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 5292 out of 5292 | elapsed:  1.4min finished


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=63,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [194]:
%%time
NFOLDS=5
skf = StratifiedKFold(n_splits=NFOLDS)


X_test=test.drop(id+target,axis=1).values
preds=np.zeros((X_test.shape[0],))
losses=[]
for train_index, val_index in skf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # print(X_train.shape,X_val.shape)
    clf=grid_search_cv.best_estimator_
        
    clf.fit(X_train,y_train)
    pval=clf.predict_proba(X_val)[:,1]
    loss=log_loss(y_val,pval)
    losses.append(loss)
    
    pred_test=clf.predict_proba(X_test)[:,1]
    preds+=pred_test
print(np.mean(losses))
preds=preds/NFOLDS
sub=pd.DataFrame()
sub["uid"]=test.reset_index()["uid"]

sub["target"]= preds
sub.head()
sub.to_csv("submit_hyp_opt1.csv",index=None)

0.31330859367867764
CPU times: user 889 ms, sys: 0 ns, total: 889 ms
Wall time: 888 ms
